# CHAPTER 13 Introduction to Modeling Libraries in Python（Python中建模库的介绍）

这一章回顾一下之间pandas的一些特性，希望能在我们处理数据的时候有所帮助。然后会简要介绍两个很有用的建模工具：statsmodels和scikit-learn。


# 13.1 Interfacing Between pandas and Model Code（pandas与建模代码间的交互）

一个通常的工作流程中，在建模之前，会用pandas来加载数据并清理。模型开发过程中，一个很重要的部分就是特征工程（feature engineering），指的是通过数据变换或分析，从原始数据中提取出对建模有用的信息。之前介绍的聚合（aggregation）和GroupBy就经常用于特征工程。

至于什么样才是好的特征工程，这就超出了本书的范围。这里会简单介绍如何在数据处理与建模之间切换。

连接pandas和其他一些分析库的点，通常是Numpy数组。要想把一个DataFrame变为Numpy数组，使用.values属性：

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5], 
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.], 
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [4]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

变回DataFrame的方法是，传入一个二维ndarray，并指定列名：

In [5]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


.values属性最好用于同质的数据，即数据类型都是数值型。如果有异质的数据，结果会变为python对象：

In [6]:
df3 = data.copy()

In [7]:
df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [8]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

对于一些模型，我们可能希望使用列中的一部分数据。建议使用loc，然后用values进行索引：

In [9]:
model_cols = ['x0', 'x1']

In [10]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

一些库对于pandas的支持非常好：能自动把DataFrame转换为numpy，并把模型的参数名字作为输出的列名。对于其他的一些库，就必须要自己手动操作了。

在第十二章里，我们学习了pandas的Categorical数据类型和pandas.get_dummies函数。假设我们的数据集中有一个非数值列：

In [11]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


如果想要哑变量来代替category这一列，我们可以创建哑变量，去除category列，然后把结果合并起来：

In [13]:
dummies = pd.get_dummies(data.category, prefix='category')
dummies

,category_a,category_b
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1


In [14]:
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


在不同的统计模型上使用哑变量有一些细微的不同。当我们有更很多非数值型列的时候，使用Patsy的话会更简单易用一些。关于Patsy的内容会在下一节进行介绍。

# 13.2 Creating Model Descriptions with Patsy（利用Patsy创建模型描述）

Patsy是一个python库，用于描述统计模型（尤其是线性模型），方法是通过一个叫做公式语法（formula syntax）的字符串来描述。这种公式语法的灵感来源于R和S语言中的公式语法。
Patsy的公式是有特殊格式的字符串，像下面这样：

    y ~ x0 + x1
    
这种a + b的语法并不代表将a和b相加，而是代表为模型创建的设计矩阵的术语（terms in the design matrix）。patsy.dmatrices函数，取一个公式字符串和一个数据集（可以使DataFrame或dict），然后为线性模型产生设计矩阵：

In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.], 
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [4]:
import patsy

In [5]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

我们得到：

In [6]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [7]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

这些Patsy DesignMatrix实例是Numpy的ndarrays，附有额外的元数据（metadata）:

In [8]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [9]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

我们可能奇怪X中的Intercept是从哪里来的。这其实是线性模型的一个惯例，比如普通最小二乘回归法（ordinary least squares regression）。我们可以去掉这个截距（intercept），通过加添术语`+0`给模型：

In [10]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

这种Patsy对象可以直接传入一个算法，比如numpy.linalg.lstsq，来进行普通最小二乘回归的计算：

In [9]:
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond=None)

In [10]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [11]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

# 1 Data Transformations in Patsy Formulas（Patsy公式的数据变换）

我们可以把python和Patsy公式混合起来。当评估公式的时候，库会尝试找到封闭域中的公式：

In [14]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [15]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

一些常用的变量变换，包括标准化（standardizing (平均值0，方差1）和中心化（减去平均值）。Patsy有内建的函数可以做到这些：

In [16]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [17]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

作为建模的一部分，我们可能会在一个数据及上训练模型，然后在另一个数据及上评价模型。当使用中心化或标准化这样的转换时，我们必须注意，必须用模型在新数据集上做预测。这叫做状态变换（stateful transformations）。因为我们必须用原本在训练集上得到的平均值和标准差，用在新的数据集上。

通过保存原先数据集中的信息，patsy.build_design_matrices函数能把变换用在新的数据集上：

In [18]:
new_data = pd.DataFrame({
        'x0': [6, 7, 8, 9], 
        'x1': [3.1, -0.5, 0, 2.3],
        'y': [1, 2, 3, 4]})

In [19]:
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

因为加号在Patsy公式中不代表加法，如果想要把两个列通过名字相加，必须把他们用I函数包起来：

In [20]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

Patsy有一些其他的内建转换，得来patsy.builtins模块里。更多的信息请参考文档。

Categorical数据有特殊的类用于变换，下面进行介绍。

# 2 Categorical Data and Patsy（Categorical数据和Patsy）

非数值型数据可以通过很多种方式变为一个模型设计矩阵。这个话题很大，这里只做简单介绍。

当我们在Patsy公式中使用非数值术语时，这些类型数据默认会被转换为哑变量。如果有截距，一个层级上的截距会被舍弃，防止出现共线性：

In [21]:
data = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'], 
                     'key2': [0, 1, 0, 1, 0, 1, 0, 0], 
                     'v1': [1, 2, 3, 4, 5, 6, 7, 8],
                     'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7] })

In [22]:
y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

如果从模型中舍弃截距，每个类型的列会被包含在模型设计矩阵中：

In [23]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

数值型列可以通过C函数，变为类型列：

In [24]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

当我们在一个模型中使用多个类型术语时，会变得更复杂一些，之前用`key1:key2`的形式来包含有交集的术语，这种方法可以用于使用多个术语，例如，一个方法分析模型（analysis of variance (ANOVA) models）：

In [25]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [27]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [28]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

Patsy还提供一些其他转换类型数据的方案，包括按特定顺序来变换。具体的可以查看文档。

# 13.3 Introduction to statsmodels（statsmodels简介）

[statsmodels](http://www.statsmodels.org/stable/index.html)是一个有很多统计模型的python库，能完成很多统计测试，数据探索以及可视化。它也包含一些经典的统计方法，比如贝叶斯方法和一个机器学习的模型。

statsmodels中的模型包括：

- 线性模型（linear models），广义线性模型（generalized linear models），鲁棒线性模型（robust linear models）
- 线性混合效应模型（Linear mixed effects models）
- 方差分析(ANOVA)方法（Analysis of variance (ANOVA) methods）
- 时间序列处理（Time series processes）和状态空间模型（state space models）
- 广义矩估计方法（Generalized method of moments）

接下来我们用一些statsmodels中的工具，并了解如何使用Patsy公式和pandas DataFrame进行建模。

# 1 Estimating Linear Models（估计线性模型）

statsmodels中的线性模型大致分为两种：基于数组的（array-based），和基于公式的（formula-based）。调用的模块为：

  

In [12]:
import statsmodels.api as sm 
import statsmodels.formula.api as smf

为了演示如何使用，我们对一些随机数据生成一个线性模型：

In [13]:
import numpy as np
import pandas as pd

In [14]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size
    return mean + np.sqrt(variance) * np.random.randn(size)
    
# For reproducibility 
np.random.seed(12345)

N = 100 
X = np.c_[dnorm(0, 0.4, size=N), 
          dnorm(0, 0.6, size=N), 
          dnorm(0, 0.2, size=N)] 

eps = dnorm(0, 0.1, size=N) 
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [15]:
print(X.shape)
print(eps.shape)

(100, 3)
(100,)


真正的模型用的参数是beta，dnorm的功能是产生指定平均值和方差的随机离散数据，得到：

In [16]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [17]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

一个线性模型通常会有一个截距，这里我们用sm.add_constant函数添加一个截距列给X：

In [18]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

sm.OLS可以拟合（fit）普通最小二乘线性回归：

In [19]:
model = sm.OLS(y, X)

fit方法返回的是一个回顾结果对象，包含预测模型的参数和其他一些诊断数据：

In [20]:
results = model.fit()
results.params

array([0.17826108, 0.22303962, 0.50095093])

在results上调用summary方法，可能得到一些详细的诊断数据：

In [21]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Thu, 18 Jul 2019   Prob (F-statistic):                    7.44e-12
Time:                        15:57:07   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.001       0.073       0.283
x2             0.2230      0.046      4.818      0.000       0.131       0.315
x3             0.5010      0.080      6.237      0.000       0.342       0.660
==============================================================================
Omnibus:                        4.662   Durbin-Watson:                   2.201
Prob(Omnibus):                  0.097   Jarque-Bera (JB):                4.098
Skew:                           0.481   Prob(JB):                        0.129
Kurtosis:                       3.243   Cond. No.                         1.74
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

参数的名字通常为x1, x2，以此类推。假设所有的模型参数都在一个DataFrame里：

In [20]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y

In [21]:
data.head()

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


现在我们可以使用statsmodels formula API（公式API）和Patsy的公式字符串：

In [22]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [23]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

可以看到statsmodel返回的结果是Series，而Series的索引部分是DataFrame的列名。当我们使用公式和pandas对象的时候，不需要使用add_constant。

如果得到新的数据，我们可以用预测模型的参数来进行预测：

In [24]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

其他一些分析、诊断、可视化工具可以自己多尝试。

# 2 Estimating Time Series Processes（预测时序过程）

statsmodels中的另一个类是用于时间序列分析的，其中有自动回归处理（autoregressive processes）， 卡尔曼滤波（Kalman filtering），状态空间模型（state space models），多元回归模型（multivariate autoregressive models）。

让我们用自动回归结果和噪音模拟一个时间序列数据：



In [25]:
init_x = 4

import random
values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [30]:
values[:6]

[4,
 4,
 1.8977509636904242,
 0.086865262206104243,
 -0.57694691325353353,
 -0.49950238023089472]

这种数据有AR(2)结构（two lags，延迟两期），延迟参数是0.8和-0.4。当我们拟合一个AR模型，我们可能不知道延迟的期间是多少，所以可以在拟合时设一个比较大的延迟数字：

In [31]:
MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)

结果里的预测参数，第一个是解决，之后是两个延迟（lags）：

In [32]:
results.params

array([-0.00616093,  0.78446347, -0.40847891, -0.01364148,  0.01496872,
        0.01429462])

关于模型的更多细节，可以查看文档。

# 13.4 Introduction to scikit-learn（scikit-learn简介）


scikit-learn是一个被广泛使用的python机器学习工具包。里面包含了很多监督式学习和非监督式学习的模型，可以实现分类，聚类，预测等任务。

虽然scikit-learn并没有和pandas深度整合，但在训练模型之前，pandas在数据清洗阶段能起很大作用。

> 译者：构建的机器学习模型的一个常见流程是，用pandas对数据进行查看和清洗，然后把处理过的数据喂给scikit-learn中的模型进行训练。

这里用一个经典的kaggle比赛数据集来做例子，泰坦尼克生还者数据集。加载训练集和测试集：

In [22]:
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')

In [23]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


statsmodels和scikit-learn通常不能应付缺失值，所以我们先检查一下哪些列有缺失值：

In [24]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [25]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

对于这样的数据集，通常的任务是预测一个乘客最后是否生还。在训练集上训练模型，在测试集上验证效果。

上面的Age这一列有缺失值，这里我们简单的用中位数来代替缺失值：

In [26]:
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

对于Sex列，我们将其变为IsFemale，用整数来表示性别：

In [27]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [28]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsFemale
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


接下来决定一些模型参数并创建numpy数组：

In [29]:
predictors = ['Pclass', 'IsFemale', 'Age']

In [30]:
X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values

In [31]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [32]:
y_train[:5]

array([0, 1, 1, 1, 0])

这里我们用逻辑回归模型（LogisticRegression）：

In [33]:
from sklearn.linear_model import LogisticRegression

ModuleNotFoundError: No module named 'sklearn'

In [15]:
model = LogisticRegression()

然后是fit方法来拟合模型：

In [16]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

在测试集上进行预测，使用model.predict:

In [18]:
y_predict = model.predict(X_test)
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

如果我们有测试集的真是结果的话，可以用来计算准确率或其他一些指标：

    (y_true == y_predcit).mean()
    
实际过程中，训练模型的时候，经常用到交叉验证（cross-validation），用于调参，防止过拟合。这样得到的预测效果会更好，健壮性更强。

交叉验证是把训练集分为几份，每一份上又取出一部分作为测试样本，这些被取出来的测试样本不被用于训练，但我们可以在这些测试样本上验证当前模型的准确率或均方误差（mean squared error），而且还可以在模型参数上进行网格搜索（grid search）。一些模型，比如逻辑回归，自带一个有交叉验证的类。LogisticRegressionCV类可以用于模型调参，使用的时候需要指定正则化项C，来控制网格搜索的程度：

In [19]:
from sklearn.linear_model import LogisticRegressionCV

In [20]:
model_cv = LogisticRegressionCV(10)

In [21]:
model_cv.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

如果想要自己来做交叉验证的话，可以使用cross_val_score函数，可以用于数据切分。比如，把整个训练集分为4个不重叠的部分：

In [22]:
from sklearn.model_selection import cross_val_score

In [25]:
model = LogisticRegression(C=10)
model

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
scores = cross_val_score(model, X_train, y_train, cv=4)
scores

array([ 0.77232143,  0.80269058,  0.77027027,  0.78828829])

默认的评价指标每个模型是不一样的，但是可以自己指定评价函数。交差验证的训练时间较长，但通常能得到更好的模型效果。